In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib tk
import pyxem as pxm
import numpy as np
import hyperspy.api as hs
import pymatgen as pmg
from matplotlib import pyplot as plt

In [33]:
dp = pxm.load('/home/pc494/Documents/data/GaN_intro/GaN_indent.blo')

In [34]:
dp = pxm.ElectronDiffraction(dp)

In [37]:
dp.metadata

├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

In [6]:
#for calibration
#dp.axes_manager
dp.plot()

In [8]:
#dp = dp.inav[301:325-20,65:89-20]
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]));

A Jupyter Widget

In [25]:
dp.set_calibration(1/8)
dp.plot()

In [9]:
Ga = pmg.Element("Ga")
N = pmg.Element("N")
lattice = pmg.Lattice.hexagonal(3.189,5.185)

In [10]:
plt.close('all')

In [11]:
u = (1/8)*2
structure = pmg.Structure.from_spacegroup(168,lattice, [Ga,Ga,N,N], [[0, 0, 0],[2/3,1/3,1/2],[0,0,0.5+u],[2/3,1/3,u]])

structure.make_supercell([3,3,1],to_unit_cell=False)

In [12]:
d = structure.as_dict()
plan_view_points = []
sp = []
for x in d['sites']:
    plan_view_points.append(np.abs((np.round(x['xyz'],2))))
    sp.append(x['label']=='Ga')

sp = np.asarray([float(x) for x in sp])
pvp = np.asarray(plan_view_points)
print(np.unique(pvp[:,2]))
for layer in np.unique(pvp[:,2]):
    pv_layer = pvp[pvp[:,2]==layer]
    sp_layer = sp[pvp[:,2]==layer]
    col = 'b'
    if np.sum(sp_layer) > 0:
        col = 'r'
    f = plt.figure()
    ax = f.add_subplot(1, 1, 1)
    ax.set_aspect(aspect='equal')
    plt.title(str(layer))
    plt.scatter(pv_layer[:,0],pv_layer[:,1],marker='o',c=col)
    plt.show()

[ 0.    1.3   2.59  3.89]


In [13]:
plt.close('all')

In [14]:
from pyxem.utils.sim_utils import equispaced_so3_grid

alpha = equispaced_so3_grid(90,180,90,5)
rot_list = alpha.tolist()

In [15]:
alphas = np.linspace(1.10,1.3,5)
betas = np.linspace(1.85,1.90,1)
gammas = np.linspace(-2.0,-1.7,1)

In [16]:
rot_list = np.array(np.meshgrid(alphas, betas, gammas)).T.reshape(-1,3).tolist()

In [26]:
edc = pxm.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pxm.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib["GaN"] = (structure, rot_list)
calib = 8

In [27]:
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=calib,
                                            reciprocal_radius=1.4,
                                            representation='euler')

In [28]:
from pyxem.indexation_generator import IndexationGenerator
indexer = IndexationGenerator(dp,library)
match_results = indexer.correlate(n_largest=5)

A Jupyter Widget

In [29]:
match_results.inav[0,0].data #are these angles in degrees?

array([[ 0.       ,  1.1      ,  1.85     , -2.       ,  0.0282874],
       [ 0.       ,  1.25     ,  1.85     , -2.       ,  0.0282874],
       [ 0.       ,  1.2      ,  1.85     , -2.       ,  0.0282874],
       [ 0.       ,  1.3      ,  1.85     , -2.       ,  0.0282874],
       [ 0.       ,  1.15     ,  1.85     , -2.       ,  0.0282874]])

In [30]:
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

peaks= match_results.map(peaks_from_best_template,phase=['GaN'],library=library,inplace=False)
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
#mmx = np.add(mmx/calib,72*(0.007603))
#mmy = np.add(mmy/calib,72*(0.007603))

A Jupyter Widget

In [31]:
dp.plot(cmap='viridis') 
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='yellow',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=True)

In [64]:
peaks.inav[0,0].data

array([[ 0.92590655,  0.92990703],
       [ 1.06440986, -0.58004552],
       [ 0.84629445, -0.09959332],
       [ 0.46295328,  0.46495352],
       [ 0.07961211,  1.02950035],
       [ 0.60145658, -1.04499904],
       [ 0.38334117, -0.56454684],
       [ 0.        ,  0.        ],
       [-0.38334117,  0.56454684],
       [-0.76668234,  1.12909368],
       [-0.46295328, -0.46495352],
       [-0.84629445,  0.09959332],
       [-1.22963561,  0.66414016],
       [-0.92590655, -0.92990703],
       [-1.30924772, -0.36536019]])

In [ ]:
# Is the problem that our rotation isn't through the middle of the direct beam?